<a href="https://colab.research.google.com/github/ykitaguchi77/Whisper_transcription/blob/main/Whisper_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Whisper Fine Tuning**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
sound_path = "/content/drive/MyDrive/Deep_learning/Transcription/2023-07-16_15-47-33.WAV"

In [3]:
!pip install pydub

In [8]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os
import shutil

sound_path = "/content/drive/MyDrive/Deep_learning/Transcription/2023-07-16_15-47-33.WAV"
source = AudioSegment.from_file(sound_path, format="wav")

chunks = split_on_silence(source,
                          min_silence_len = 100,
                          silence_thresh = -40,
                          keep_silence = 200)

outputDirname="/content/drive/MyDrive/Deep_learning/Transcription/data"

if os.path.exists(outputDirname):
  shutil.rmtree(outputDirname)

os.makedirs(outputDirname,exist_ok=True)

fileList=[]
current_chunk = None

for i, chunk in enumerate(chunks):
    if current_chunk is None:
      current_chunk = chunk
    else:
      current_chunk+=chunk

    if len(current_chunk)/1000>30:
      num_of_chunk = int(len(current_chunk)/30000)
      for j in range(num_of_chunk+1):
        outFilePath = f'{outputDirname}/out_{i + 1}_{j + 1}.wav'
        current_chunk[j*30000:(j+1)*30000].export(outFilePath, format="wav")
        fileList.append(outFilePath)
      current_chunk = None

In [4]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ddb5k1tr
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ddb5k1tr
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 820.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.7 MB/s eta 0:00:

In [ ]:
!pip install datasets

In [7]:
from datasets import Dataset,Audio
import whisper
from whisper.audio import N_FRAMES, pad_or_trim, log_mel_spectrogram
from whisper.tokenizer import get_tokenizer

model = whisper.load_model("large")
datasets = Dataset.from_dict({"audio": fileList}).cast_column("audio", Audio())

predict_data = []
for i, f in enumerate(fileList):
  mel = log_mel_spectrogram(f)
  segment = pad_or_trim(mel, N_FRAMES).to(model.device).to(torch.float16)
  result = model.decode(segment)
  tokenizer = get_tokenizer(multilingual=True, language="ja", task="transcribe")
  outputText = tokenizer.decode(result.tokens)
  predict_data.append({**datasets[i], "sentence":outputText})

100%|██████████████████████████████████████| 2.88G/2.88G [00:26<00:00, 115MiB/s]


NameError: name 'fileList' is not defined

In [ ]:
import gspread
from google.auth import default
from google.colab import auth

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

spread_sheet_dir_id = getoutput("xattr -p 'user.drive.id' " + "'" + outputDirname[:-5] + "'")
ss = gc.create('Transcription_Dataset', folder_id=spread_sheet_dir_id)

ss.append_row(['url', 'colab_path','sampling_rate','correct','whisper'])

rows = []
for data in predict_data:
  file_id = getoutput("xattr -p 'user.drive.id' " + "'" +  data["audio"]["path"] + "'")
  url = f"https://drive.google.com/file/d/{file_id}"
  colab_path = data["audio"]["path"]
  sampling_rate = data["audio"]["sampling_rate"]
  whisper_text = data["sentence"]
  rows.append([url, colab_path, sampling_rate, "", whisper_text])

ss.values_append("シート1",{"valueInputOption":"USER_ENTERED"},{"values":rows})